<a href="https://colab.research.google.com/github/G544/galinanana_ml/blob/main/happy_whale/Happy_whales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"galinanana","key":"ab98fce8fd3a978d35813c91eda8156c"}'}

In [2]:

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"galinanana","key":"ab98fce8fd3a978d35813c91eda8156c"}
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c happy-whale-and-dolphin
! git clone https://github.com/G544/galinanana_ml.git

100% 57.7G/57.7G [04:43<00:00, 219MB/s]
100% 57.7G/57.7G [04:44<00:00, 218MB/s]


In [58]:
import io
import pandas as pd
import zipfile
from pathlib import Path
from sklearn.model_selection import train_test_split

# Чтение файла

In [46]:

zip = zipfile.ZipFile('/content/happy-whale-and-dolphin.zip')
names = (zip.namelist())
train_names = []
label = []
test_names = []
for item in names:
  comp = item.split('_')[0]
  if comp == 'test':
    test_names.append(Path(item))
  elif comp == 'train':
    train_names.append(Path(item))
  elif comp == 'train.csv':
    label = item

label = pd.read_csv(io.BytesIO(zip.read(label)))
label = label.set_index('image')
label = label['individual_id']

# Деление выборки

In [59]:
import random
train_val_files = sorted(train_names, key=lambda A: random.random())
train_val_labels = [label[(path.name)] for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

ValueError: ignored

In [61]:
! git status

fatal: not a git repository (or any of the parent directories): .git


In [60]:
train_val_labels

['fb492f141125',
 '6f632b846891',
 'b61c15e89c3c',
 'b0d69bb109f7',
 'db59c6b576c9',
 '3116029ec53a',
 '4a4671af9741',
 'c4deab707116',
 '110942914010',
 '8cfe17a71664',
 'daa006a682c7',
 '90d51ec83be4',
 '910aff333c1b',
 '49f9246ea28b',
 '7355eee87d75',
 '290f46e5130c',
 'b91c12ffc2b0',
 'ce873781a2c4',
 '06518e8bc9e7',
 '7cb0a479c64d',
 '82c322b0938e',
 '60008f293a2b',
 '9b9f1eabedab',
 '0f72263cd384',
 '1a455afda685',
 'd5d9b19035c4',
 '8fda3e3bdc5f',
 '10bf999f2fed',
 '0a229e510bd2',
 '812be36c2aef',
 '19c2bfaa0819',
 'da65f819073c',
 'dba4e482f0ad',
 '46fb363aecd6',
 'd975000283fa',
 '31f748b822f4',
 '7af019614856',
 'a3605aa0335e',
 '814597f508b8',
 'c48ffb700e3f',
 '37c7aba965a5',
 '2d46f4a0355b',
 '0c7efa8795f5',
 '2fdb3a09dc9c',
 'fd63ed3877af',
 'ee0ca4cf0faf',
 '9afcb84d0057',
 '1044d8b29af0',
 '27037997d3a2',
 '50e4963b2441',
 '9712ec1d3f84',
 'b857caa6d4a2',
 '12f2a4406d7e',
 '19fbb960f07d',
 '208b91b1ca2b',
 'ce6e37904aa4',
 '1de4cee1f2cb',
 'e56c835a7858',
 '6fde4fcebb13

# Черновики


In [34]:
import io
import pandas as pd
label_daat = pd.read_csv(io.BytesIO(zip.read(label)))

In [35]:
label_daat = pd.read_csv(io.BytesIO(zip.read(label)))
label_daat = label_daat.set_index('image')
label_try = label_daat['individual_id']

In [44]:
lab = label_try[[train_names[0].name, train_names[5].name]].to_numpy()

In [45]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(lab)


LabelEncoder()

```python

if self.mode != 'test':  
            lab_file = io.BytesIO(archive.read(lab_file))
            df = pd.read_csv(lab_file, header=None)
            self.labels = df.to_numpy()[[int(Path(path).name) for path in self.files]]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

```


In [26]:
len(train_names)

51033

In [22]:
label_daat[label_daat['image'] == str(train_names[0]).split('/')[1]]

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9


In [8]:
from galinanana_ml.happy_whale import zipDataset, fit_model

In [ ]:
test = ZIPSimpsonsDataset(archive,train_files, lab_file=label_file, mode='train') 